## Import Libraries. Upload Dataset.

In [50]:
import pandas as pd
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

In [39]:
df = pd.read_csv("C:/Users/chris/Documents/Datasets/BaseballR/2021/2021merged.csv", dtype={61:"str",151:"str",152:"str",153:"str",154:"str",155:"str",156:"str"})
df = df[(df.pitcher == 669302)]

## Filter out columns not being referenced.

In [158]:
#df.columns.values

In [51]:
logan = df[["game_date.x", "at_bat_number", "pitch_number", "pitch_name", "release_pos_x", "release_pos_y", "release_pos_z"]]

In [52]:
logan.head()

,game_date.x,at_bat_number,pitch_number,pitch_name,release_pos_x,release_pos_y,release_pos_z
32990,2021-09-30,1.0,1.0,4-Seam Fastball,-1.41,53.08,6.12
32991,2021-09-30,1.0,2.0,4-Seam Fastball,-1.20,53.03,6.22
32992,2021-09-30,1.0,3.0,4-Seam Fastball,-1.33,53.02,6.00
32993,2021-09-30,1.0,4.0,4-Seam Fastball,-1.10,53.03,6.12
32994,2021-09-30,1.0,5.0,4-Seam Fastball,-1.35,52.91,6.04


## Build headers and dropdown menus for plotly dash app.

In [54]:
logan.pitch_name.unique()

array(['4-Seam Fastball', 'Slider', 'Changeup', 'Knuckle Curve'],
      dtype=object)

In [214]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Logan Gilbert, 2021", style={"text-align" : "center"}),
    html.H2("Release Point Variation by Pitch, Game Date", style={"text-align" : "center"}),
    
    html.Div(
        children=[
            dcc.Dropdown(id="sel_pitch",
                         options=[
                             {"label": "4-Seam Fastball", "value": "4-Seam Fastball"},
                             {"label": "Slider", "value": "Slider"},
                             {"label": "Changeup", "value": "Changeup"},
                             {"label": "Knuckle Curve", "value": "Knuckle Curve"}
                         ],
                         multi=False,
                         value="4-Seam Fastball",
                         style={"width": 240}
                        )],
        style = {"display": "flex", "align-items": "center", "justify-content": "center"}
    ),
    html.Div(
        children=[
            dcc.Dropdown(id="sel_date",
                         options=[
                             {"label": "Before 8-21", "value": "Before"},
                             {"label": "After 8-21", "value": "After"},
                             {"label": "All Dates", "value": "All Dates"}
                         ],
                         multi=False,
                         value="All Dates",
                         style={"width": 240}
                        )],
        style = {"display": "flex", "align-items": "center", "justify-content": "center"}
    ),
    html.Div(
        dcc.Graph(id="graph", figure={}),
        style = {"display": "flex", "align-items": "center", "justify-content": "center"}
    )
])

## Build graph being referenced by our app.

In [215]:
@app.callback(
    Output(component_id="graph", component_property="figure"),
    [Input(component_id="sel_pitch", component_property="value"),
     Input(component_id="sel_date", component_property="value")]
)

def update_graph(option_pitch, option_date):
    print(option_pitch)
    print(option_date)
    
    dff = df.copy()
    dff = dff[(dff.pitch_name == option_pitch)]
    
    if option_date == "Before":
        dff = dff[(dff["game_date.x"] <= "2021-08-21")]
    elif option_date == "After":
        dff = dff[(dff["game_date.x"] > "2021-08-21")]
    
    x = dff.release_pos_x
    y = dff.release_pos_y
    z = dff.release_pos_z
    
    fig = go.Figure(data=[go.Scatter3d(
        x=x, 
        y=y, 
        z=z, 
        mode="markers",
        marker=dict(
            size=6,
            color=z,
            colorscale="Viridis",
            opacity=0.8
        )
    )])
    
    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1, y=-1, z=.5)
    )
    
    fig.update_layout(
        autosize=False,
        width=1200,
        height=800,
        scene = dict(xaxis_title="Horizontal Distance (ft)",
                     yaxis_title="Distance from Home (ft)",
                     zaxis_title="Height (ft)",
                     xaxis=dict(range=[-3,0]),
                     yaxis=dict(range=[52,55]),
                     zaxis=dict(range=[5.5, 6.6])
                    ),
        scene_camera = camera
    )
    return fig

app.run_server(mode='inline')

4-Seam Fastball
All Dates
4-Seam Fastball
All Dates
4-Seam Fastball
All Dates
4-Seam Fastball
All Dates
4-Seam Fastball
All Dates
4-Seam Fastball
Before
4-Seam Fastball
All Dates
